In [1]:
import re
import numpy as np
import multiprocessing as mp
import sys
import pandas as pd
import QuantLib as ql
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from scipy.stats import pearsonr
import seaborn as sns

In [2]:
df_IF = pd.read_pickle('IF.pkl')
df_IH = pd.read_pickle('IH.pkl')
df_IC = pd.read_pickle('IC.pkl')
df_IM = pd.read_pickle('IM.pkl')

In [3]:
df_IF.head(3)

,basis,contract_close,index_close,basis_chg,index_close_chg
2022-05-23 09:30:00,-17.969,4063.0,4080.969,NaN,NaN
2022-05-23 09:31:00,-21.600,4052.2,4073.800,-3.631,-7.169
2022-05-23 09:32:00,-22.509,4044.4,4066.909,-0.909,-6.891


In [4]:
class order_manager:
    def __init__(self):
        self.open_orders = {}
        self.closed_orders = []
        self.order_id = 1
        self.money = 1000000
        self.fee_rate = 0.0023
        #self.balance = self.money
        self.equity = self.money
        self.contract_size = 300

    def open_order(self, code, position, price, open_time, lot):

        order_id = f"{self.order_id:06d}"
        self.order_id += 1
        
        value = price*self.contract_size*lot
        open_fee = value*self.fee_rate
        
        order_info = {
            'order_id':order_id,
            'code':code,
            'open_price':price,
            'open_time':open_time,
            'position':position,
            'lot':lot, 
            'value':value, 
            'open_fee':open_fee,
            #'balance':balance,
            #'equity':equity
        } 

        self.open_orders[order_id] = order_info
        #print(f"开仓成功 - ID: {order_id} | 方向: {position} | 品种: {code} | 价格: {price} | 手数: {lot}")
        return order_id
    
    def close_order(self, order_id, price, close_time):
        
        if order_id not in self.open_orders:
            print(f"平仓失败 - 未找到订单ID: {order_id}")
            return None
    
        order = self.open_orders.pop(order_id) #取出并删除指定键对应的值！

        if order['position'] == 1:
            profit = (price - order['open_price'])*order['lot']*self.contract_size
        elif order['position'] == -1:
            profit = -(price - order['open_price'])*order['lot']*self.contract_size
            
        close_fee = price * order['lot'] * self.contract_size * self.fee_rate
        net_profit = profit # - order['open_fee'] - close_fee
        self.equity = self.equity + net_profit
        
        closed_order = {
            **order,
            'close_price': price,
            'close_time': close_time,
            'profit': profit,
            'fee': order['open_fee'] + close_fee,
            'net_profit': net_profit,
            'equity' : self.equity,
            'position' : order['position'],
            'holding_period': (close_time - order['open_time']).total_seconds() / 3600
        }
        
        self.closed_orders.append(closed_order)
        
        #print(f"平仓成功 - ID: {order_id} | 方向: {order['position']} | "
             # f":手数: {order['lot']} | 开仓价: {order['open_price']} | "
              #f"平仓价: {price} | 净利润: {net_profit:.2f}")
        
        return closed_order

In [5]:
manager = order_manager()

In [6]:
def fixed_time_close(code, start_time, end_time, raw_data,window = 30):
    start_time = datetime.strptime(start_time, '%Y%m%d')
    end_time = datetime.strptime(end_time, '%Y%m%d')
    
    data = raw_data.loc[start_time:end_time].copy()
    
    data['basis_chg_MA'] = data['basis_chg'].rolling(window=window).mean()
    data['basis_chg_std'] =  data['basis_chg'].rolling(window=window).std()
    data['Upper'] = data['basis_chg_MA'] + 2 * data['basis_chg_std']
    data['Lower'] = data['basis_chg_MA'] - 2 * data['basis_chg_std']
    
    print(data)
    
    time_list = list(data.index)
    for time in time_list:
#注意list(manager.open_orders.items())返回的是许多元组构成的列表，每个元组为（id,字典）
        for order_id, order_info in list(manager.open_orders.items()):
            if time >= order_info['open_time'] + timedelta(minutes=15):
                manager.close_order(
                    order_id=order_id,
                    price=data.loc[time]['contract_close'],  # 使用当前价格平仓
                    close_time=time
                )
                
        basis = data.loc[time]['basis']
        basis_chg = data.loc[time]['basis_chg']
        upper = data.loc[time]['Upper']
        lower = data.loc[time]['Lower']
        
        if basis_chg < lower:
            long_order = manager.open_order(
                code = code,
                position = -1,
                price = data.loc[time]['contract_close'],
                open_time = time,
                lot = 1
            )
            
        if basis_chg > upper:
            short_order = manager.open_order(
                code = code,
                position = 1,
                price = data.loc[time]['contract_close'],
                open_time = time,
                lot = 1
            )

In [7]:
fixed_time_close('IF','20250101','20250520',df_IF,30)

                      basis  contract_close  index_close  basis_chg  \
2025-01-02 09:30:00 -10.816          3921.0     3931.816      3.095   
2025-01-02 09:31:00  -0.886          3925.0     3925.886      9.930   
2025-01-02 09:32:00  -4.809          3921.0     3925.809     -3.923   
2025-01-02 09:33:00  -5.696          3919.8     3925.496     -0.887   
2025-01-02 09:34:00  -1.575          3918.8     3920.375      4.121   
...                     ...             ...          ...        ...   
2025-05-16 14:56:00 -40.623          3849.6     3890.223     -0.227   
2025-05-16 14:57:00 -40.137          3848.6     3888.737      0.486   
2025-05-16 14:58:00 -40.587          3848.2     3888.787     -0.450   
2025-05-16 14:59:00 -41.587          3847.2     3888.787     -1.000   
2025-05-16 15:00:00 -43.086          3846.0     3889.086     -1.499   

                     index_close_chg  basis_chg_MA  basis_chg_std     Upper  \
2025-01-02 09:30:00           -3.095           NaN            NaN   

In [8]:
closed_orders_df = pd.DataFrame(manager.closed_orders)
closed_orders_df

,order_id,code,open_price,open_time,position,lot,value,open_fee,close_price,close_time,profit,fee,net_profit,equity,holding_period
0,000001,IF,3879.0,2025-01-02 10:22:00,-1,1,1163700.0,2676.510,3882.0,2025-01-02 10:37:00,-900.0,5355.090,-900.0,999100.0,0.250000
1,000002,IF,3889.6,2025-01-02 10:52:00,-1,1,1166880.0,2683.824,3884.6,2025-01-02 11:07:00,1500.0,5364.198,1500.0,1000600.0,0.250000
2,000003,IF,3881.2,2025-01-02 11:13:00,1,1,1164360.0,2678.028,3878.8,2025-01-02 11:28:00,-720.0,5354.400,-720.0,999880.0,0.250000
3,000004,IF,3878.4,2025-01-02 13:05:00,1,1,1163520.0,2676.096,3868.8,2025-01-02 13:20:00,-2880.0,5345.568,-2880.0,997000.0,0.250000
4,000005,IF,3867.2,2025-01-02 13:23:00,1,1,1160160.0,2668.368,3850.8,2025-01-02 13:38:00,-4920.0,5325.420,-4920.0,992080.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,000920,IF,3845.0,2025-05-16 10:27:00,1,1,1153500.0,2653.050,3848.4,2025-05-16 10:42:00,1020.0,5308.446,1020.0,1059400.0,0.250000
920,000921,IF,3849.4,2025-05-16 11:26:00,1,1,1154820.0,2656.086,3849.2,2025-05-16 13:01:00,-60.0,5312.034,-60.0,1059340.0,1.583333
921,000922,IF,3848.0,2025-05-16 11:30:00,-1,1,1154400.0,2655.120,3849.2,2025-05-16 13:01:00,-360.0,5311.068,-360.0,1058980.0,1.516667
922,000923,IF,3841.4,2025-05-16 14:07:00,-1,1,1152420.0,2650.566,3847.4,2025-05-16 14:22:00,-1800.0,5305.272,-1800.0,1057180.0,0.250000


In [9]:
def get_backtest(df):
    df['close_time'].str[:10]
    #daily_returns = df.groupby(df['close_time'])['net_profit'].sum() / df.groupby(df['close_time'].str[:8])['equity'].first()
    #return daily_returns

In [10]:
manager.money + sum(closed_orders_df['net_profit'])

1059999.999999999

In [11]:
len(manager.closed_orders)

924

In [12]:
positive_count = sum(1 for order in manager.closed_orders if order["net_profit"] > 0)
positive_count

437

In [13]:
win_rate = positive_count / len(manager.closed_orders)
win_rate

0.47294372294372294